# Room acoustics example.

Workshop this example a bit, make it so that there is some kind of classification pipeline 

In [5]:
import numpy as np

from datumlib import collect, Datum
import datumlib as dl

from itertools import product
from dataclasses import dataclass

from dataclasses import field


@dataclass(frozen=True)
class TimeSeries(Datum[np.ndarray]):
    sample_rate: float = field(kw_only=True)

    @classmethod
    def create(cls, data: np.ndarray, sample_rate: float, *, tags={}):
        return cls(data, sample_rate=sample_rate, tags=tags)

    @property
    def n_samples(self) -> int:
        return len(self.data)

    @property
    def duration(self) -> float:
        return self.n_samples / self.sample_rate

    @property
    def is_valid(self) -> bool:
        return True if len(self.data.shape) == 1 else False


# Example usage
t = TimeSeries.create(
    np.array([1, 2, 3]),
    44100,
    tags={
        "id": "1234-4321",
        "signal-strength": 10,
    },
)

dl.display_datum(t)

TimeSeries
├── data: array([1, 2, 3])
├── sample_rate: 44100
└── tags
    ├── id: '1234-4321'
    └── signal-strength: 10

Generate some fake IRs with source and receiver data and 'known' T60.

In [6]:
rng = np.random.default_rng(42)
sample_rate = 32_000

sources: dict[str, tuple[float, float, float]] = {
    "S1: Corner": (0.0, 0.0, 0.0),
    "S2: Listener": (0.5, 0.5, 0.5),
}

receivers: dict[str, tuple[float, float, float]] = {
    "R1": (0.1, 0.2, 0.3),
    "R2": (0.6, 0.7, 0.5),
    "R3": (0.4, 0.2, 0.2),
    "R4": (0.1, 0.5, 0.7),
    "R5": (0.4, 0.5, 1.5),
}


def generate_rir(length=sample_rate * 8, rt60=0.6, noise=1e-4):
    t = np.arange(length) / sample_rate

    # amplitude decay consistent with RT60 definition
    decay = np.exp(-6.91 * t / rt60)

    rir = decay * rng.standard_normal(length)
    rir[0] = 1.0
    rir += noise * rng.standard_normal(length)

    return rir


# make decaying signals
signals = []
for (src_id, src_loc), (rec_id, rec_loc) in product(sources.items(), receivers.items()):
    rt60 = rng.uniform(0.3, 1.1)
    rir = generate_rir(rt60=rt60)

    # append metadata to each thing
    signals.append(
        TimeSeries(
            rir,
            sample_rate=sample_rate,
            tags={
                "rt60_true": rt60,
                "source_id": src_id,
                "source_loc": src_loc,
                "receiver_id": rec_id,
                "receiver_loc": rec_loc,
            },
        )
    )


In [7]:
# collect into `MonoCollection`
collection = collect(*signals, tags={"dataset": "synthetic_rooms"})
dl.display_collection(collection)

╭──────────────────────────────────────────────── DatumCollection ────────────────────────────────────────────────╮
│ Collection Tags                                                                                                 │
│ dataset  synthetic_rooms                                                                                        │
│ entries (10)                                                                                                    │
│ ├── 0 TimeSeries                                                                                                │
│ │   ├── data: array([ 1.00010289e+00,  7.50417518e-01,  9.40179589e-01, ...,                                    │
│ │   │           1.25343830e-05, -7.41785290e-05, -1.60626260e-04], shape=(256000,))                             │
│ │   ├── sample_rate: 32000                                                                                      │
│ │   └── tags                                                                                                    │
│ │       ├── rt60_true: 0.9191648388447706                                                                       │
│ │       ├── source_id: 'S1: Corner'                                                                             │
│ │       ├── source_loc: (0.0, 0.0, 0.0)                                                                         │
│ │       ├── receiver_id: 'R1'                                                                                   │
│ │       └── receiver_loc: (0.1, 0.2, 0.3)                                                                       │
│ ├── 1 TimeSeries                                                                                                │
│ │   ├── data: array([ 1.00009599e+00,  2.17418877e+00, -7.37825948e-01, ...,                                    │
│ │   │          -2.68496914e-05,  2.22796028e-04,  8.57640219e-05], shape=(256000,))                             │
│ │   ├── sample_rate: 32000                                                                                      │
│ │   └── tags                                                                                                    │
│ │       ├── rt60_true: 0.7764788075309272                                                                       │
│ │       ├── source_id: 'S1: Corner'                                                                             │
│ │       ├── source_loc: (0.0, 0.0, 0.0)                                                                         │
│ │       ├── receiver_id: 'R2'                                                                                   │
│ │       └── receiver_loc: (0.6, 0.7, 0.5)                                                                       │
│ ├── 2 TimeSeries                                                                                                │
│ │   ├── data: array([ 9.99925705e-01, -7.21054033e-01, -1.62685991e+00, ...,                                    │
│ │   │          -3.74306106e-05, -1.02940989e-04,  7.22164644e-05], shape=(256000,))                             │
│ │   ├── sample_rate: 32000                                                                                      │
│ │   └── tags                                                                                                    │
│ │       ├── rt60_true: 1.0655089704677436                                                                       │
│ │       ├── source_id: 'S1: Corner'                                                                             │
│ │       ├── source_loc: (0.0, 0.0, 0.0)                                                                         │
│ │       ├── receiver_id: 'R3'                                                                                   │
│ │       └── receiver_loc: (0.4, 0.2, 0.2)                                                                       │
│ ├── 3 TimeSeries                                      

Define a preprocessing pipeline

In [8]:
from datumlib import Datum
from datumlib import datum_util as dutil
from datumlib import PipelineDict


@dl.cmap
def truncate(d: TimeSeries) -> TimeSeries:
    trunc_sec = 2
    return d.with_data(d.data[: trunc_sec * d.sample_rate])


# lift data function to operate on datum inputs
@dl.cmap
@dutil.over_data
def normalize_func(x: np.ndarray) -> np.ndarray:
    return x / np.max(np.abs(x))


@dl.cmap
def tag_processed(d: Datum) -> Datum:
    return d.add_tags("processed", True)


# define a preprocessing pipeline. This will sequentially process
# the signals in `collection` and return an updated collection
pipeline = PipelineDict(
    {
        "normalize": normalize_func,
        "truncate": truncate,
        "mark_processed": tag_processed,
    }
)

# Process collection via pipeline and visualize progress
collection = pipeline(collection, progress_meter=True)
dl.display_collection(collection)


Processing DatumCollection:  67%|██████▋   | 2/3 [00:00<00:00, 101.92step/s]


TypeError: Datum.add_tags() takes 2 positional arguments but 3 were given

Ideas, add way to compare collections, or interact (like modulate one collection with another by matching metadata field or sth)